# Ionic Conductivity Simulation Tutorial
This notebook demonstrates how to simulate and analyze the ionic conductivity of Li$^+$ ions in LLZO using GPUMD and Python tools. It covers simulation setup, MSD calculation, diffusion coefficient extraction, conductivity calculation, and activation energy estimation.

## Required Dependencies
- GPUMD (for MD simulation)
- Python 3.x
- numpy
- matplotlib
- scipy
- GPUMDkit (for analysis and plotting)

Install Python packages with:
```python
!pip install numpy matplotlib scipy
```
GPUMDkit: [https://github.com/zhyan0603/GPUMDkit](https://github.com/zhyan0603/GPUMDkit)
GPUMD: [https://gpumd.org/](https://gpumd.org/)

## Step 1: Prepare Input Files
- `model.xyz`: Atomic structure of LLZO.
- `nep.txt`: NEP potential file (see below for example).

If you need to train a NEP model, use the NEP package and your training dataset. Example training command:
```bash
nep train --input training_data.extxyz --output nep.txt
```
For this tutorial, a pre-trained NEP file is provided.

In [ ]:
# Check input files
import os
for fname in ['model.xyz', 'nep.txt']:
    print(f'{fname}:', 'Found' if os.path.exists(fname) else 'Missing')

## Step 2: GPUMD Simulation Setup
Create a GPUMD input file (e.g., `run.in`) for NPT simulation and MSD calculation. Example:
```
potential ../nep.txt
velocity    1000
ensemble    npt_mttk temp 1000 1000 aniso 0 0
run        50000
ensemble    npt_mttk temp 1000 1000 aniso 0 0
compute_msd 10 5000 group 0 0
dump_thermo 100
dump_exyz 1000000
run        1000000
```
Run the simulation with GPUMD:
```bash
gpumd
```
This generates `msd.out` and `thermo.out` files.

## Step 3: Analyze MSD Data
The MSD quantifies Li$^+$ ion movement. Columns in `msd.out`:
1. Time (ps)
2. MSD$_x$ (Å$^2$)
3. MSD$_y$ (Å$^2$)
4. MSD$_z$ (Å$^2$)
5. SDC$_x$ (Å$^2$/ps)
6. SDC$_y$ (Å$^2$/ps)
7. SDC$_z$ (Å$^2$/ps)

In [ ]:
# Plot MSD
import numpy as np
import matplotlib.pyplot as plt
data = np.loadtxt('msd.out')
time = data[:, 0]
msd = data[:, 1:4].sum(axis=1)
plt.plot(time, msd)
plt.xlabel('Time (ps)')
plt.ylabel('MSD (Å$^2$)')
plt.title('Mean Squared Displacement of Li$^+$')
plt.show()

## Step 4: Calculate Diffusion Coefficient
Fit the linear region of MSD vs. time to get the slope $k$ (Å$^2$/ps).
The diffusion coefficient $D$ is:
$D = k / (2N)$, where $N=3$ for 3D diffusion.

In [ ]:
# Fit linear region to get D
start = int(len(time) * 0.1)
end = int(len(time) * 0.4)
k, _ = np.polyfit(time[start:end], msd[start:end], 1)
D = k / (2 * 3)
print(f'Diffusion coefficient D = {D:.3e} Å^2/ps')

## Step 5: Calculate Ionic Conductivity
Use the Nernst-Einstein relation:
$igma = n q^2 D / (k_B T)$
Where:
- $n$: number density of ions
- $q$: elementary charge
- $D$: diffusion coefficient
- $k_B$: Boltzmann constant
- $T$: temperature (K)

In [ ]:
# Calculate ionic conductivity
import scipy.constants as consts
N_ion = 3584  # Example: number of Li ions
volume = 143800.291  # Å^3, from thermo.out
T = 1000  # K
vol_cm3 = volume * 1e-24
n = N_ion / vol_cm3
q = consts.e
kB = consts.k
D_cm2_s = D * 1e-4  # Convert Å^2/ps to cm^2/s
sigma = n * q**2 * D_cm2_s / (kB * T)
print(f'Ionic conductivity σ = {sigma:.3e} S/cm')

## Step 6: Estimate Activation Energy
Run simulations at multiple temperatures, calculate $igma$ for each, and fit $n(igma T)$ vs $1/T$ to get activation energy $E_a$.

In [ ]:
# Example: Arrhenius plot for activation energy
temps = np.array([1000, 1100, 1200])
sigma_Ts = np.array([sigma1, sigma2, sigma3])  # Replace with calculated values
x = 1000 / temps
y = np.log(sigma_Ts * temps)
slope, intercept = np.polyfit(x, y, 1)
Ea = -slope * consts.k * 1000 / consts.e
print(f'Activation energy Ea = {Ea:.3f} eV')
plt.plot(x, y, 'o')
plt.plot(x, slope * x + intercept, '-')
plt.xlabel('1000/T (1/K)')
plt.ylabel('ln(σT)')
plt.title('Arrhenius Plot')
plt.show()

## References
- Zihan Yan, Zheyong Fan and Yizhou Zhu, [arXiv:2504.15925](https://doi.org/10.48550/arXiv.2504.15925)
- Zihan Yan and Yizhou Zhu, [Chem. Mater. 2024, 36, 23, 11551–11557](https://doi.org/10.1021/acs.chemmater.4c02454)